In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from diagnostics.inventory import ModelInventoryBuilder, QueryBuilder
from diagnostics.inventory import load_metric_csv, get_model_type

In [2]:
# prompt to specify directory containing config files
dataset_name = "mirror-mouse-debug"
artifacts_path = f"/home/jovyan/grid_artifacts/{dataset_name}/" # "/media/mattw/behavior/results/pose-estimation/mirror-mouse/2022-11-17"
df_save_path = f"/home/jovyan/results/{dataset_name}" # "/media/mattw/behavior/results/pose-estimation/mirror-mouse"
assert os.path.exists(artifacts_path) and os.path.exists(df_save_path)
rng_seeds_list =  ["0", "1", "2", "3", "4"]
train_frames_list = ["75", "1"]

In [3]:
# define the metrics for labeled data and video data, these very per dataset
if dataset_name == "fly" or dataset_name == "fly-debug" or dataset_name == "ibl-pupil":
    # there are no multiple views here
    labeled_metrics = ["pixel_error", "pca_singleview_error"]
    video_metrics = ["temporal_norm", "pca_singleview_error"]
else:
    labeled_metrics = ["pixel_error", "pca_singleview_error", "pca_multiview_error"]
    video_metrics = ["temporal_norm", "pca_singleview_error", "pca_multiview_error"]



In [4]:
model_inventory = ModelInventoryBuilder(artifacts_path)
total_df = model_inventory.build_dframe()

Building model registry from 10 configs...


100%|██████████| 10/10 [00:00<00:00, 34.99it/s]


In [5]:
query_builder = QueryBuilder(total_df)
query_builder.add_query("training.rng_seed_data_pt", "in", rng_seeds_list)
query_builder.add_query("training.train_frames", "in", train_frames_list)
query_builder.add_query("model.backbone", "==", "resnet50") # NOTE: for debug

# query_builder.add_query("model.losses_to_use", "in", ["[]", "[pca_multiview]"]) # trying to grab both unimodal_mse and supervised. note no quotes inside brackets per loss
query_builder.add_timestamp_query("2023-01-06", "2023-01-07") # works

# query_builder.add_query("losses.pca_multiview.log_weight", ">", 4.) 

total_df_queried = total_df.query(query_builder.combine_queries("and"))
total_df_queried.shape

`training.rng_seed_data_pt` == '0' or `training.rng_seed_data_pt` == '1' or `training.rng_seed_data_pt` == '2' or `training.rng_seed_data_pt` == '3' or `training.rng_seed_data_pt` == '4'
`training.train_frames` == '75' or `training.train_frames` == '1'


(5, 91)

In [6]:
def update_col_names(df):
    old_names_0 = df.columns.levels[0]
    new_names_0 = {}
    for n in old_names_0:
        new_name = n if n.find("Unnamed") == -1 else "set"
        new_names_0[n] = new_name
    old_names_1 = df.columns.levels[1]
    new_names_1 = {}
    for n in old_names_1:
        new_name = n if n.find("Unnamed") == -1 else ""
        new_names_1[n] = new_name
    df = df.rename(columns=new_names_0, level=0)
    df = df.rename(columns=new_names_1, level=1)
    return df

def get_video_names(file_list):
    video_names = []
    for file in file_list:
        if "_pca_singleview_error" in file:
            continue
        elif "_pca_multiview_error" in file:
            continue
        elif "_temporal_norm" in file:
            continue
        else:
            video_names.append(file.replace(".csv", ""))
    return video_names

def add_model_metadata(df, model, levels):
    updates = {
        "model_path": model["path"],
        "rng_seed_data_pt": model["training.rng_seed_data_pt"],
        "train_frames": model["training.train_frames"],
        "model_type": get_model_type(model),
    }
    for key, val in updates.items():
        # always put the key at the top level of a multi-index
        # fill out remaining levels with empty strings
        acc_str = (key,)
        for _ in range(1, levels):
            acc_str += ("",)
        df.loc[:, acc_str] = val

def load_confidence_csv(filename):
    df = pd.read_csv(filename, header=[1, 2], index_col=0)
    vals = df.to_numpy()
    likelihoods = vals[:, 2::3]
    splits = vals[:, -1]
    keypoint_names = df.columns.get_level_values(0)[2::3]
    df_confs = pd.DataFrame(likelihoods, columns=keypoint_names, index=df.index)
    df_confs["mean"] = df_confs.mean(axis=1)
    if vals.shape[1] % 3 == 1:
        # labeled data
        df_confs["set"] = splits
    df_confs["metric"] = "confidence"
    return df_confs


In [7]:
def return_chunked_df(df, chunk_list):
    # assert that df has video_name column and that it has one value in it 
    assert "video_name" in df.columns and len(df["video_name"].unique()) == 1, "df must have video_name column and only one value in it"
    # get video name
    video_name = df["video_name"].unique()[0]
    # loop over chuncks, copy slices, and add to df. modify video_name column to include start and end of chunk
    df_curr_chunks = []
    for chunk in chunk_list:
        vid_chunk_name = "%s_%d_%d" % (video_name, chunk[0], chunk[1])
        df_chunk = df.iloc[chunk[0]:chunk[1]].copy() # copy is important to avoid assinging to a slice of a view
        df_chunk["video_name"] = vid_chunk_name
        df_curr_chunks.append(df_chunk) # append to list
    # overwrite df with new df
    return pd.concat(df_curr_chunks) # concat list to a single df for this single video. contains 5x2000 frame chunks.

In [8]:
total_df_queried.shape

(5, 91)

In [10]:
# loop over rows of df, load predictions_pixel_error.csv, predictions_pca_singleview_error.csv
df_labeled_preds = []
df_labeled_metrics = []
df_video_preds = []
df_video_metrics = []
mirror_mouse_chunks = [[0, 2000], [10000, 12000], [20000, 22000], [25000, 27000]] # start, end for each chunk in each video of mirror_mouse
for i, model in tqdm(total_df_queried.iterrows()):
    
    # --------------------
    # labeled predictions
    # --------------------
    df_labeled_preds_curr = []
    for distribution_type in ["InD", "OOD"]:
        if distribution_type == "InD":
            filename = os.path.join(model["path"], "predictions.csv")
        else:
            filename = os.path.join(model["path"], "predictions_new.csv")
        df = pd.read_csv(filename, header=[1, 2], index_col=0)
        df = update_col_names(df)
        df.loc[:, ("distribution", "")] = distribution_type
        if distribution_type == "OOD":
            df.loc[:, ("set", "")] = "test"
        df_labeled_preds_curr.append(df)
    df_labeled_preds_curr = pd.concat(df_labeled_preds_curr)
    add_model_metadata(df_labeled_preds_curr, model, levels=2)
    df_labeled_preds.append(df_labeled_preds_curr)
    
    # --------------------
    # labeled metrics
    # --------------------
    df_labeled_metrics_curr = []
    for distribution_type in ["InD", "OOD"]:
        # load precomputed errors
        for metric_name in labeled_metrics: # defined above per dataset
            if distribution_type == "InD":
                filename = os.path.join(model["path"], "predictions_%s.csv" % metric_name)
            else:
                filename = os.path.join(model["path"], "predictions_new_%s.csv" % metric_name)
            if os.path.isfile(filename):
                df = load_metric_csv(
                    filename, metric_name, None, pd_kwargs={"header": [0], "index_col": 0})
                df["distribution"] = distribution_type
                if distribution_type == "OOD":
                    df["set"] = "test"
                df_labeled_metrics_curr.append(df)
        # load confidences from predictions
        if distribution_type == "InD":
            filename = os.path.join(model["path"], "predictions.csv")
        else:
            filename = os.path.join(model["path"], "predictions_new.csv")
        df = load_confidence_csv(filename)
        df["distribution"] = distribution_type
        if distribution_type == "OOD":
            df["set"] = "test"
        df_labeled_metrics_curr.append(df)
    df_labeled_metrics_curr = pd.concat(df_labeled_metrics_curr)
    add_model_metadata(df_labeled_metrics_curr, model, levels=1)
    df_labeled_metrics.append(df_labeled_metrics_curr)

    # --------------------
    # video predictions
    # --------------------
    video_names = get_video_names(os.listdir(os.path.join(model["path"], "video_preds")))
    df_video_preds_curr = []
    for video_name in video_names:
        filename = os.path.join(model["path"], "video_preds", "%s.csv" % video_name)
        df = pd.read_csv(filename, header=[1, 2], index_col=0)
        df["video_name"] = video_name
        # a unique block for mirror-mouse to get more vid slices
        if dataset_name == "mirror-mouse" or dataset_name == "mirror-mouse-debug":
            # overwrite df with new df
            df = return_chunked_df(df, mirror_mouse_chunks)
            
        df_video_preds_curr.append(df) # concat different vids to one list
    df_video_preds_curr = pd.concat(df_video_preds_curr) # list into df (override name)
    add_model_metadata(df_video_preds_curr, model, levels=2)  # in-place
    df_video_preds.append(df_video_preds_curr)

    # # --------------------
    # # video metrics
    # # --------------------
    video_names = get_video_names(os.listdir(os.path.join(model["path"], "video_preds")))
    # keep only video names that include the 3 specific dates (hack because we have too much data)
    df_video_metrics_curr = []
    for video_name in video_names:
        # load precomputed metrics
        for metric_name in video_metrics: # defined above per dataset
            filename = os.path.join(
                model["path"], "video_preds", "%s_%s.csv" % (video_name, metric_name))
            df = load_metric_csv(
                filename, metric_name, None, pd_kwargs={"header": [0], "index_col": 0})
            df["video_name"] = video_name
            # if mirror mouse dataset take multiple chunks
            if dataset_name == "mirror-mouse" or dataset_name == "mirror-mouse-debug":
                df = return_chunked_df(df, mirror_mouse_chunks)
            df_video_metrics_curr.append(df)
        # load confidences from predictions
        filename = os.path.join(model["path"], "video_preds", "%s.csv" % video_name)
        df = load_confidence_csv(filename)
        df["video_name"] = video_name
        if dataset_name == "mirror-mouse" or dataset_name == "mirror-mouse-debug":
            df = return_chunked_df(df, mirror_mouse_chunks)
        df_video_metrics_curr.append(df)
    df_video_metrics_curr = pd.concat(df_video_metrics_curr)
    add_model_metadata(df_video_metrics_curr, model, levels=1)  # in-place
    df_video_metrics.append(df_video_metrics_curr)
    

5it [00:10,  2.07s/it]


In [11]:
# concat all dfs
df_labeled_preds = pd.concat(df_labeled_preds)
df_labeled_metrics = pd.concat(df_labeled_metrics)
df_video_preds = pd.concat(df_video_preds)
df_video_metrics = pd.concat(df_video_metrics)

In [15]:
# df_video_preds.shape[0] / 5 / 5 / 2000 / 4 # 5 models, 5 videos, 2000 frames, 4 chunks
# df_video_metrics.shape[0] / 5 / 5 / 2000 / 4 / 4 # number of metrics for mirror mouse
#df_video_preds.shape[0] / 40 / 5 / 2000 / 4

In [12]:
# save out dfs
df_labeled_preds.to_parquet(os.path.join(df_save_path, "%s_labeled_preds.pqt" % dataset_name))
df_labeled_metrics.to_parquet(os.path.join(df_save_path, "%s_labeled_metrics.pqt" % dataset_name))
df_video_preds.to_parquet(os.path.join(df_save_path, "%s_video_preds.pqt" % dataset_name))
df_video_metrics.to_parquet(os.path.join(df_save_path, "%s_video_metrics.pqt" % dataset_name))

In [18]:
df_video_metrics["metric"].unique()

array(['temporal_norm', 'pca_singleview_error', 'pca_multiview_error',
       'confidence'], dtype=object)

In [19]:
df_video_preds.shape

(1600000, 56)

In [ ]:
750